In [1]:
import os
import random
import math
from sklearn.metrics import mean_squared_error
import numpy as np
from matplotlib import pyplot as plt
import pandas as pd
import seaborn as sn
from sklearn.decomposition import PCA
from sklearn.preprocessing import MinMaxScaler, StandardScaler
import pmdarima as pm 
from sklearn.model_selection import train_test_split
from pmdarima.pipeline import Pipeline

In [2]:
os.getcwd()

'C:\\Users\\niles\\project\\project_work - 1214\\notebooks'

In [2]:
# Read data:
#os.chdir("..")
df = pd.read_csv("data.csv")

In [3]:
df.head()

,Time,air_pressure[mmHg],air_temperature[degree celcius],relative_humidity[%],wind_speed[M/S],solar_irridiation[W/m²],total_cloud_cover[from ten],electricity_demand_values[kw],heat_demand_values[kw]
0,2010-12-01 00:00:00,729.7,25.0,85.0,5.0,0,no clouds,289.567557,432.940036
1,2010-12-01 01:00:00,729.4,27.8,77.0,7.0,0,no clouds,260.168520,473.935901
2,2010-12-01 02:00:00,728.9,33.3,62.0,7.0,0,2/10–3/10.,247.273585,483.278761
3,2010-12-01 03:00:00,731.6,32.2,62.0,2.0,0,5/10.,257.955878,545.921252
4,2010-12-01 04:00:00,732.6,22.8,96.0,3.0,0,2/10–3/10.,258.255081,550.526112


In [5]:
len(df)

70080

In [6]:
df = df.dropna()

In [7]:
len(df)

68792

In [8]:
# Rename Columns:
col_mapping = {"Time": "date",
            "air_pressure[mmHg]": "air_pressure",
            "air_temperature[degree celcius]": "air_temperature",
            "relative_humidity[%]": "relative_humidity",
            "wind_speed[M/S]": "wind_speed",
            "solar_irridiation[W/m²]": "solar_irridiation",
            "total_cloud_cover[from ten]": "total_cloud_cover",
            "electricity_demand_values[kw]": "electricity",
            "heat_demand_values[kw]": "heat"}
df = df.rename(columns = col_mapping)

In [9]:
# Change ts to datetime and set as index:
df["date"] = pd.to_datetime(df["date"],format='%Y-%m-%d')
df.head()

,date,air_pressure,air_temperature,relative_humidity,wind_speed,solar_irridiation,total_cloud_cover,electricity,heat
0,2010-12-01 00:00:00,729.7,25.0,85.0,5.0,0,no clouds,289.567557,432.940036
1,2010-12-01 01:00:00,729.4,27.8,77.0,7.0,0,no clouds,260.168520,473.935901
2,2010-12-01 02:00:00,728.9,33.3,62.0,7.0,0,2/10–3/10.,247.273585,483.278761
3,2010-12-01 03:00:00,731.6,32.2,62.0,2.0,0,5/10.,257.955878,545.921252
4,2010-12-01 04:00:00,732.6,22.8,96.0,3.0,0,2/10–3/10.,258.255081,550.526112


In [10]:
len(df)

68792

In [11]:
df.head()

,date,air_pressure,air_temperature,relative_humidity,wind_speed,solar_irridiation,total_cloud_cover,electricity,heat
0,2010-12-01 00:00:00,729.7,25.0,85.0,5.0,0,no clouds,289.567557,432.940036
1,2010-12-01 01:00:00,729.4,27.8,77.0,7.0,0,no clouds,260.168520,473.935901
2,2010-12-01 02:00:00,728.9,33.3,62.0,7.0,0,2/10–3/10.,247.273585,483.278761
3,2010-12-01 03:00:00,731.6,32.2,62.0,2.0,0,5/10.,257.955878,545.921252
4,2010-12-01 04:00:00,732.6,22.8,96.0,3.0,0,2/10–3/10.,258.255081,550.526112


In [12]:
df1 = df[["date",'electricity']]

In [13]:
df1 = df1.set_index("date")
df1.head()

,electricity
date,
2010-12-01 00:00:00,289.567557
2010-12-01 01:00:00,260.168520
2010-12-01 02:00:00,247.273585
2010-12-01 03:00:00,257.955878
2010-12-01 04:00:00,258.255081


In [14]:
x = df1['electricity'].values.reshape(-1, 1) #returns a numpy array
min_max_scaler = StandardScaler()
x_scaled = min_max_scaler.fit_transform(x)
df1['electricity'] = x_scaled

In [15]:
df1.head()

,electricity
date,
2010-12-01 00:00:00,-0.437728
2010-12-01 01:00:00,-0.560519
2010-12-01 02:00:00,-0.614377
2010-12-01 03:00:00,-0.569760
2010-12-01 04:00:00,-0.568511


In [16]:
davg_df = df1.groupby(pd.Grouper(freq='D')).mean()
davg_df.head()

,electricity
date,
2010-12-01,-0.159240
2010-12-02,-0.270442
2010-12-03,-0.122860
2010-12-04,0.095403
2010-12-05,0.213953


In [17]:
len(davg_df)

2920

In [18]:
dftrain, dftest = train_test_split(davg_df, test_size=0.01*3)

In [19]:
len(dftest)

88

In [20]:
dftrain, dfvali = train_test_split(dftrain, test_size=0.2)

In [21]:
len(dftrain)

2265

In [22]:
len(dfvali)

567

In [23]:
# best
random.seed(0)

In [24]:

pipeline = Pipeline([
    ("model", pm.AutoARIMA(test='adf',
                           start_p=3, start_q=3, max_p=5, max_q=5, start_P=0, start_Q=0, max_P=5, max_Q=10, m=12, stepwise=True, seasonal=True, information_criterion='aic', trace=True, d=1, D=1, error_action='warn', 
                           suppress_warnings=True, random_state=20, n_fits=100))
])
'''
pipeline = Pipeline([
    ("model",pm.AutoARIMA(test='adf',
                           max_p=3, max_q=3, m=7,
                           start_P=1,start_Q=1, seasonal=True,
                           d=1, D=1, trace=True,
                           error_action='ignore',  
                           suppress_warnings=True, 
                           stepwise=False))
])
'''
pipeline.fit(dftrain)

Performing stepwise search to minimize aic
 ARIMA(3,1,3)(0,1,0)[12]             : AIC=inf, Time=3.76 sec
 ARIMA(0,1,0)(0,1,0)[12]             : AIC=6749.605, Time=0.09 sec
 ARIMA(1,1,0)(1,1,0)[12]             : AIC=5566.399, Time=0.37 sec
 ARIMA(0,1,1)(0,1,1)[12]             : AIC=inf, Time=2.17 sec
 ARIMA(1,1,0)(0,1,0)[12]             : AIC=6105.830, Time=0.10 sec
 ARIMA(1,1,0)(2,1,0)[12]             : AIC=5291.113, Time=0.82 sec
 ARIMA(1,1,0)(3,1,0)[12]             : AIC=5170.381, Time=1.50 sec
 ARIMA(1,1,0)(4,1,0)[12]             : AIC=5096.162, Time=2.65 sec
 ARIMA(1,1,0)(5,1,0)[12]             : AIC=5026.978, Time=6.38 sec
 ARIMA(1,1,0)(5,1,1)[12]             : AIC=inf, Time=44.50 sec
 ARIMA(1,1,0)(4,1,1)[12]             : AIC=inf, Time=32.47 sec
 ARIMA(0,1,0)(5,1,0)[12]             : AIC=5635.864, Time=4.27 sec
 ARIMA(2,1,0)(5,1,0)[12]             : AIC=4736.434, Time=7.05 sec
 ARIMA(2,1,0)(4,1,0)[12]             : AIC=4801.654, Time=3.07 sec
 ARIMA(2,1,0)(5,1,1)[12]             

Pipeline(steps=[('model',
                 AutoARIMA(D=1, d=1, error_action='warn', m=12, max_P=5,
                           max_Q=10, n_fits=100, random_state=20, start_P=0,
                           start_Q=0, start_p=3, start_q=3, test='adf',
                           trace=True))])

In [47]:
##### univariate arima model predict electricity_demand_values  
# best
#Arima_model=pm.auto_arima(dftrain['electricity_demand_values'],start_p=3, start_q=3, max_p=5, max_q=5, start_P=0, start_Q=0, max_P=5, max_Q=10, m=12, stepwise=True, seasonal=True, information_criterion='aic', trace=True, d=1, D=1, error_action='warn', suppress_warnings=True, random_state=20, n_fits=100)

#Arima_model=pm.auto_arima(dftrain['electricity_demand_values'],start_p=3, start_q=3, max_p=5, max_q=6, start_P=0, start_Q=4, max_P=6, max_Q=10, m=12, stepwise=True, seasonal=True, information_criterion='aic', trace=True, d=1, D=1, error_action='warn', suppress_warnings=True, random_state=20, n_fits=100)

#Arima_model=pm.auto_arima(dftrain['electricity_demand_values'],start_p=3, start_q=3, max_p=5, max_q=6, start_P=0, start_Q=4, max_P=6, max_Q=10, m=12, stepwise=True, seasonal=True, information_criterion='bic', trace=True, d=1, D=1, error_action='warn', suppress_warnings=True, n_fits=100)

#Arima_model=pm.auto_arima(dftrain['electricity_demand_values'],start_p=3, start_q=3, max_p=5, max_q=6, start_P=1, start_Q=4, max_P=6, max_Q=10, m=12, stepwise=True, seasonal=True, information_criterion='bic', trace=True, d=1, D=1, error_action='warn', suppress_warnings=True, n_fits=100)


In [37]:
Sample_predictions= pipeline.predict_in_sample(exogenous=None, return_conf_int=False, alpha=0.05, inverse_transform=True)
Actuals = pd.Series.to_numpy(dftrain['electricity'])
Residuals = Actuals-Sample_predictions

C:\ProgramData\Anaconda3\lib\site-packages\pmdarima\arima\arima.py:692: UserWarning: As of version 1.5.0 'typ' is no longer a valid arg for predict. In future versions this will raise a TypeError.
  warnings.warn("As of version 1.5.0 'typ' is no longer a valid "


In [38]:
Residuals

array([ 0.48599307, -0.13418045,  0.71193533, ..., -0.72912341,
       -0.81633549,  0.17406534])

In [39]:
forecast, conf = pipeline.predict(n_periods=567, return_conf_int=True)
forecast

array([-0.56601924, -0.36996223, -0.29905121, -0.31845445, -0.40946456,
       -0.67394054, -0.36616808, -0.46126265, -0.2577197 , -0.61999893,
       -0.14052735, -0.79325413, -0.74197541, -0.43335642, -0.23008721,
       -0.30678344, -0.40296327, -0.67150268, -0.39765848, -0.46286516,
       -0.32796555, -0.63582001, -0.37513274, -0.82500818, -0.86003993,
       -0.50756676, -0.28930281, -0.40690682, -0.42195317, -0.67119176,
       -0.54408756, -0.54351971, -0.42502196, -0.71864467, -0.49086419,
       -0.87991141, -0.90706332, -0.63842334, -0.42673067, -0.54515104,
       -0.40507011, -0.74303251, -0.59483153, -0.59784716, -0.53154749,
       -0.81128366, -0.4993565 , -0.94984522, -1.08072246, -0.69319344,
       -0.52997136, -0.72980235, -0.52365011, -0.87178929, -0.87503195,
       -0.68582989, -0.66889945, -0.89059175, -0.73207158, -1.03128196,
       -1.15125935, -0.86515967, -0.68103518, -0.76313726, -0.53482318,
       -0.94636647, -0.93226991, -0.81412992, -0.77257174, -1.08

In [40]:
Actuals = pd.Series.to_numpy(dfvali['electricity'])
Actuals

array([-2.88433222e-01,  5.78174191e-01, -3.26843503e-01,  3.58785264e-01,
       -2.46639204e-01,  5.75046374e-01, -9.31610390e-02, -4.47932319e-01,
       -2.81070475e-01, -6.46399201e-01, -2.57446150e-01,  3.94035884e-01,
        1.00651070e-01, -2.56795832e-01, -7.21103469e-01,  1.86840789e-01,
       -8.69330044e-02,  4.64363598e-01,  1.56835086e+00, -5.02699185e-01,
        2.53527543e-02, -2.39328569e-01, -1.29184304e-01, -1.21200701e-01,
       -5.97817414e-01, -2.34923323e-01,  4.24083059e-01, -1.43014517e-01,
        3.89621037e-01,  4.13381681e-01,  2.17197093e-01, -5.37292686e-01,
       -3.56671455e-01, -5.31162961e-01, -5.33064122e-01,  2.40751322e-01,
       -4.43316974e-01,  3.11900663e-01,  1.15293973e-01,  6.41942140e-02,
        4.12782029e-01,  1.13770336e-02,  9.83676412e-02, -5.55617052e-01,
        1.79716775e-01,  7.76641007e-03, -4.40478393e-01,  2.20414471e-01,
       -5.06781060e-01,  4.95757143e-01,  1.88395311e-01, -1.97512673e-01,
       -3.28695798e-01,  

In [44]:
mseVali = mean_squared_error(Actuals, forecast)


In [45]:
forecasttrain, _ = pipeline.predict(n_periods=2265, return_conf_int=True)
forecasttrain

array([ -0.56601924,  -0.36996223,  -0.29905121, ..., -20.00314524,
       -19.93282611, -19.86672153])

In [46]:
Actualstrain = pd.Series.to_numpy(dftrain['electricity'])
Actualstrain

array([ 0.48599307,  0.35181247,  1.06374788, ..., -0.6955204 ,
       -0.5168299 , -0.57657115])

In [47]:
msetrain = mean_squared_error(Actualstrain, forecasttrain)
msetrain

135.63488892230467

In [48]:
forecasttest, conf = pipeline.predict(n_periods=88, return_conf_int=True)
forecasttest

array([-0.56601924, -0.36996223, -0.29905121, -0.31845445, -0.40946456,
       -0.67394054, -0.36616808, -0.46126265, -0.2577197 , -0.61999893,
       -0.14052735, -0.79325413, -0.74197541, -0.43335642, -0.23008721,
       -0.30678344, -0.40296327, -0.67150268, -0.39765848, -0.46286516,
       -0.32796555, -0.63582001, -0.37513274, -0.82500818, -0.86003993,
       -0.50756676, -0.28930281, -0.40690682, -0.42195317, -0.67119176,
       -0.54408756, -0.54351971, -0.42502196, -0.71864467, -0.49086419,
       -0.87991141, -0.90706332, -0.63842334, -0.42673067, -0.54515104,
       -0.40507011, -0.74303251, -0.59483153, -0.59784716, -0.53154749,
       -0.81128366, -0.4993565 , -0.94984522, -1.08072246, -0.69319344,
       -0.52997136, -0.72980235, -0.52365011, -0.87178929, -0.87503195,
       -0.68582989, -0.66889945, -0.89059175, -0.73207158, -1.03128196,
       -1.15125935, -0.86515967, -0.68103518, -0.76313726, -0.53482318,
       -0.94636647, -0.93226991, -0.81412992, -0.77257174, -1.08

In [49]:
Actualstest = pd.Series.to_numpy(dftest['electricity'])
Actualstest

array([-0.54589218, -0.42037527, -0.51036082,  0.61362476, -0.61317237,
       -0.30763293, -0.5200797 , -0.51387416,  0.09678628,  0.60551263,
       -0.6687183 , -0.52015811, -0.12820125, -0.24541617,  0.42697728,
       -0.25334244, -0.5004312 , -0.51438469, -0.55449724, -0.4440704 ,
       -0.38030461,  0.00665248, -0.27082121, -0.54930533,  0.21191174,
       -0.72121995,  0.11856004, -0.33691632,  0.48219655,  0.2097056 ,
       -0.45326051, -0.52661946, -0.14634047,  0.75834705, -0.14049582,
        0.67443888, -0.54808552, -0.15429608, -0.72214325,  2.498469  ,
       -0.49037912, -0.59452956,  0.28117604, -0.26419372,  0.1607337 ,
       -0.18296275, -0.67774189,  0.04239773, -0.22775508, -0.3971071 ,
       -0.1188432 , -0.35991256, -0.32672419, -0.02548683,  0.21007423,
       -0.52300838, -0.42136195,  0.26670321,  0.24584192,  0.14746346,
        0.24765973, -0.10630554, -0.4858824 ,  0.37032354, -0.60239183,
        0.56749275, -0.50205994,  0.68477946, -0.30644151,  0.39

In [50]:
msetest = mean_squared_error(Actualstest, forecasttest)
msetest

0.7145059063952578

In [51]:
rmse = math.sqrt(msetest)
print('RMSE: %f' % rmse)

RMSE: 0.845285


In [52]:
mse = mean_squared_error(Actuals, forecast)
mse

8.534984766097647

In [53]:
rmse = math.sqrt(mse)
print('RMSE: %f' % rmse)

RMSE: 2.921470
